# data preprocessing

In [100]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [101]:
vote=pd.read_csv("/home/22n0457/Vote.txt",names=['person id','movie id','score','weight','modified date'],sep='\t')

In [102]:
movie_df=pd.read_csv("/home/22n0457/Movie.txt",names=['movie id','name','PR_URL','IMDb_URL','theater','theater release','video status','video release','action','animation','art_foreign','classic','comedy','drama','family','horror','romance','thriller'],sep='\t',encoding='latin-1')

In [103]:
person_df=pd.read_csv("/home/22n0457/Person.txt",names=['person id','age','gender','zip code'],sep='\t',encoding='latin-1')

In [104]:
df=vote.merge(movie_df,on='movie id')
df=df.merge(person_df,on='person id')

In [105]:
df.isnull().sum()

person id                0
movie id                 0
score                    0
weight                   0
modified date            0
name                     0
PR_URL             1721450
IMDb_URL               587
theater                  0
theater release    2134226
video status         79180
video release      1825451
action                   0
animation                0
art_foreign              0
classic                  0
comedy                   0
drama                    0
family                   0
horror                   0
romance                  0
thriller                 0
age                   5619
gender              829108
zip code            725833
dtype: int64

In [106]:
df.shape

(2811983, 25)

In [107]:
#zip code theater release video release PR url imdb url has null entry so we drop that column
df.drop(columns=['PR_URL','IMDb_URL','theater release','video release','zip code'],inplace=True)

In [108]:
df['gender']=df['gender'].fillna(method='ffill')

In [109]:
df['video status']=df['video status'].fillna(method='ffill')

In [110]:
df.isnull().sum()

person id           0
movie id            0
score               0
weight              0
modified date       0
name                0
theater             0
video status        0
action              0
animation           0
art_foreign         0
classic             0
comedy              0
drama               0
family              0
horror              0
romance             0
thriller            0
age              5619
gender              0
dtype: int64

In [111]:
df.dropna(inplace=True)

In [112]:
df.isnull().sum()

person id        0
movie id         0
score            0
weight           0
modified date    0
name             0
theater          0
video status     0
action           0
animation        0
art_foreign      0
classic          0
comedy           0
drama            0
family           0
horror           0
romance          0
thriller         0
age              0
gender           0
dtype: int64

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2806364 entries, 0 to 2811982
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   person id      int64  
 1   movie id       int64  
 2   score          float64
 3   weight         float64
 4   modified date  object 
 5   name           object 
 6   theater        object 
 7   video status   object 
 8   action         int64  
 9   animation      int64  
 10  art_foreign    int64  
 11  classic        int64  
 12  comedy         int64  
 13  drama          int64  
 14  family         int64  
 15  horror         int64  
 16  romance        int64  
 17  thriller       int64  
 18  age            float64
 19  gender         object 
dtypes: float64(3), int64(12), object(5)
memory usage: 449.6+ MB


In [114]:
df.drop(columns='modified date',inplace=True)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2806364 entries, 0 to 2811982
Data columns (total 19 columns):
 #   Column        Dtype  
---  ------        -----  
 0   person id     int64  
 1   movie id      int64  
 2   score         float64
 3   weight        float64
 4   name          object 
 5   theater       object 
 6   video status  object 
 7   action        int64  
 8   animation     int64  
 9   art_foreign   int64  
 10  classic       int64  
 11  comedy        int64  
 12  drama         int64  
 13  family        int64  
 14  horror        int64  
 15  romance       int64  
 16  thriller      int64  
 17  age           float64
 18  gender        object 
dtypes: float64(3), int64(12), object(4)
memory usage: 428.2+ MB


In [116]:
df=pd.get_dummies(df,columns=['theater','video status','gender'])

In [117]:
def lowerstr(obj):
    return obj.lower()

In [118]:
df['name']=df['name'].apply(lowerstr)

In [119]:
df.reset_index(inplace=True)

In [120]:
df.drop(columns='index',inplace=True)

In [121]:
vote['sume']=[1 for i in range(vote.shape[0])]

In [122]:
vote['score']=vote['score']-0.5

In [123]:
vote.drop(columns=["weight","modified date"],inplace=True)

In [124]:
ci=[]
t=(vote.groupby(['person id']).count()['score']>100)
for j in t.index:
    if t[j]==False:
        ci.append(j)
data100=vote.pivot_table(index='movie id', columns='person id', values='score')
data100.drop(columns=ci,inplace=True)
data100.fillna(0,inplace=True)

In [125]:
np.savetxt('/home/22n0457/each100_new/data100.csv',data100, delimiter=',')

In [126]:
ci=[]
t=(vote.groupby(['person id']).count()['score']>200)
for j in t.index:
    if t[j]==False:
        ci.append(j)
data200=vote.pivot_table(index='movie id', columns='person id', values='score')
data200.drop(columns=ci,inplace=True)
data200.fillna(0,inplace=True)

In [127]:
np.savetxt('/home/22n0457/each200_new/data200.csv',data200, delimiter=',')

for more than 200 movie

# algorithm code

In [128]:
def widrowhoff(train,y,t,c):
    w=np.array([0 for i in range(len(np.transpose(train)))])
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,t+1):
        x=np.array(train[i-1])
        y_bart=np.dot(w,x)
        y_bar=[]
        for t in range(1,i+1):
            x=np.array(train[t-1])
            y_bart=np.dot(w,x)
            y_bar.append(y_bart)            
        l=float(sum([np.absolute(y[j-1]-y_bar[j-1]) for j in range(1,i+1)])/i)
        loss.append(l)
        print(f"loss in round {i} is ",l)
        w=np.subtract(w,c*(np.dot(w,x)-y[i-1])*x)
    return loss,w

In [129]:
def pranking(train,y,t):
    w=np.zeros(train.shape[1])
    b = np.zeros(len(np.unique(y)) - 1)
    b = np.append(b, np.inf)
    loss=[]
    label=range(1,len(np.unique(y))+1)
    unique_y = np.sort(np.unique(y))
    y = np.array([np.where(unique_y == labels)[0][0] + 1 for labels in y]) 
    def predict(x, w, b):
        scores = np.dot(x, w) < b
        return np.argmax(scores) + 1
    for i in range(1,t+1):
        x=train[i-1]
        y_bar=predict(x, w, b)
        if y_bar !=y[i-1]:
            y_rt=np.where(y[i-1]<=label[:-1],-1,1)
            t_rt=np.where((np.dot(x,w)-b[:-1])*y_rt <=0,y_rt,0)
            w=w+sum(t_rt)*x
            b[:-1] -= t_rt
        #predicting score
        y_bar1=np.array([predict(train[j-1],w,b) for j in range(1,i+1)])
        l = np.mean(np.abs(y[:i] - y_bar1)) #loss function
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,w,b

In [130]:
def uniformmulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    def taugenerate(h,e,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for k in e:
            t[k-1]=-(1/len(e))
        return t
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[int(r-1)],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),e,yt)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m         

In [131]:
def worstmulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,label):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        k=np.argsort([np.dot(m[j],x) for j in range(h)])[-2]
        t[k]=-1
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[int(r-1)],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),m,x,yt,label)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

In [132]:
def vimulticlassalgo(feature,y,maxiter):
    m=np.zeros((len(np.unique(y)),len(feature[0])))
    y_bar1=[]
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        for i in range(h):
            if i!=int(yt-1):
                if np.dot(m[i],x)-np.dot(m[int(yt-1)],x)>0:
                    num=np.dot(m[i],x)-np.dot(m[int(yt-1)],x)
                else:
                    num=0
                den=0
                for k in range(h):
                    if np.dot(m[k],x)-np.dot(m[int(yt-1)],x)>0:
                        den+=np.dot(m[k],x)-np.dot(m[int(yt-1)],x)
                if den==0:
                    den=1
                t[i]=-(num/den)     
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        e=[]
        for r in range(1,len(label)+1):
            if  r!=yt:
                if np.dot(m[r-1],x)>=np.dot(m[int(yt-1)],x):
                    e.append(r)
        if len(e)!=0:
            tau=taugenerate(len(label),m,x,yt)
            for k in label:
                m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

In [133]:
def mira(feature,y,maxiter):
    m=np.ones((len(np.unique(y)),len(feature[0])))
    y_bar1=[]
    loss=[]
    label=range(1,len(np.unique(y))+1)
    l2=np.sort(np.unique(y))
    def taugenerate(h,m,x,yt,ybart):
        t=[0 for i in range(h)]
        t[int(yt-1)]=1
        tau=np.minimum(0.001, (((m[int(ybart-1)]-m[int(yt-1)])*x)+1.0)/(2.0*(np.dot(x,x))))
        t[int(yt-1)]=tau
        t[int(ybart-1)]=-tau
        return t
    for i in range(len(y)):
        for j in range(len(label)):
            if y[i]==l2[j]:
                y[i]=label[j]
    for i in range(1,maxiter+1):
        x=feature[i-1]
        ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
        yt=y[i-1]
        tau=taugenerate(len(label),m,x,yt,ybart)
        for k in label:
            m[k-1]=np.add(m[k-1],tau[k-1]*x)
        y_bar1=[]       
        for t in range(1,i+1):
            x=feature[t-1]
            ybart=np.argmax([np.dot(m[j],x) for j in range(len(label))])+1
            y_bar1.append(ybart)
        l=(1/i)*sum([np.absolute(y[j-1]-y_bar1[j-1]) for j in range(1,i+1)])
        print(f"loss in round {i} is ",l)
        loss.append(l)
    return loss,m

# Experiment

In [134]:
w1_synthetic=[]
loss11=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss1,w1=widrowhoff(x,y,400,0.001)
    loss11.append(loss1)
    w1_synthetic.append(w1)

loss in round 1 is  4.0
loss in round 2 is  1.8502600000000478
loss in round 3 is  1.653090903866686
loss in round 4 is  2.327948745847082
loss in round 5 is  2.3555816426816563
loss in round 6 is  2.259179767776851
loss in round 7 is  2.1172156772356567
loss in round 8 is  2.105608947406907
loss in round 9 is  2.191767120916756
loss in round 10 is  2.1523198895034885
loss in round 11 is  1.9729395404927388
loss in round 12 is  1.9583086870504713
loss in round 13 is  2.011000327742003
loss in round 14 is  2.075695549714367
loss in round 15 is  2.0777815585175614
loss in round 16 is  2.0654367560777356
loss in round 17 is  2.0030728879999113
loss in round 18 is  1.9588772684097566
loss in round 19 is  1.9815988875359347
loss in round 20 is  1.9702042011060956
loss in round 21 is  1.931335835177112
loss in round 22 is  1.8913941987230458
loss in round 23 is  1.9024204739975337
loss in round 24 is  1.8654028390404027
loss in round 25 is  1.801947355956907
loss in round 26 is  1.7637436901

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss1.csv',loss11, delimiter=',')
np.savetxt('/home/22n0457/each100_new/w1_synthetic.csv',w1_synthetic, delimiter=',')

In [135]:
w2_synthetic=[]
b2_synthetic=[]
loss21=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss2,w2,b2=pranking(x,y,400)
    loss21.append(loss2)
    w2_synthetic.append(w2)
    b2_synthetic.append(b2)

loss in round 1 is  2.0
loss in round 2 is  1.0
loss in round 3 is  0.6666666666666666
loss in round 4 is  1.0
loss in round 5 is  2.2
loss in round 6 is  2.1666666666666665
loss in round 7 is  1.1428571428571428
loss in round 8 is  1.875
loss in round 9 is  1.4444444444444444
loss in round 10 is  1.3
loss in round 11 is  1.1818181818181819
loss in round 12 is  1.3333333333333333
loss in round 13 is  1.2307692307692308
loss in round 14 is  1.1428571428571428
loss in round 15 is  1.4666666666666666
loss in round 16 is  1.625
loss in round 17 is  1.7058823529411764
loss in round 18 is  1.7222222222222223
loss in round 19 is  1.631578947368421
loss in round 20 is  1.45
loss in round 21 is  1.8571428571428572
loss in round 22 is  1.7727272727272727
loss in round 23 is  1.826086956521739
loss in round 24 is  1.7083333333333333
loss in round 25 is  1.64
loss in round 26 is  1.5769230769230769
loss in round 27 is  1.7407407407407407
loss in round 28 is  1.7857142857142858
loss in round 29 is 

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss2.csv',loss21, delimiter=',')
np.savetxt('/home/22n0457/each100_new/w2_synthetic.csv',w2_synthetic, delimiter=',')
#np.savetxt('/home/22n0457/each100_new/b2_synthetic.csv',b2_synthetic, delimiter=',')

In [136]:
loss31=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss3,m1=uniformmulticlassalgo(x,y,400)
    loss31.append(loss3)

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.]
loss in round 6 is  [0.]
loss in round 7 is  [0.]
loss in round 8 is  [0.]
loss in round 9 is  [0.]
loss in round 10 is  [0.]
loss in round 11 is  [0.27272727]
loss in round 12 is  [0.25]
loss in round 13 is  [0.23076923]
loss in round 14 is  [0.21428571]
loss in round 15 is  [0.2]
loss in round 16 is  [0.1875]
loss in round 17 is  [0.17647059]
loss in round 18 is  [0.16666667]
loss in round 19 is  [0.47368421]
loss in round 20 is  [0.15]
loss in round 21 is  [0.14285714]
loss in round 22 is  [0.13636364]
loss in round 23 is  [0.13043478]
loss in round 24 is  [0.125]
loss in round 25 is  [0.24]
loss in round 26 is  [0.23076923]
loss in round 27 is  [0.22222222]
loss in round 28 is  [0.21428571]
loss in round 29 is  [0.10344828]
loss in round 30 is  [0.1]
loss in round 31 is  [0.09677419]
loss in round 32 is  [0.09375]
loss in round 33 is  [0.09090909]
loss in rou

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss3.csv',loss31, delimiter=',')

In [137]:
loss41=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss4,m2=worstmulticlassalgo(x,y,400)
    loss41.append(loss4)

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.]
loss in round 6 is  [0.]
loss in round 7 is  [0.]
loss in round 8 is  [0.375]
loss in round 9 is  [0.33333333]
loss in round 10 is  [0.3]
loss in round 11 is  [0.27272727]
loss in round 12 is  [0.25]
loss in round 13 is  [0.23076923]
loss in round 14 is  [0.21428571]
loss in round 15 is  [0.2]
loss in round 16 is  [0.1875]
loss in round 17 is  [0.23529412]
loss in round 18 is  [0.22222222]
loss in round 19 is  [0.52631579]
loss in round 20 is  [0.5]
loss in round 21 is  [0.38095238]
loss in round 22 is  [0.36363636]
loss in round 23 is  [0.34782609]
loss in round 24 is  [0.33333333]
loss in round 25 is  [0.32]
loss in round 26 is  [0.30769231]
loss in round 27 is  [0.33333333]
loss in round 28 is  [0.21428571]
loss in round 29 is  [0.34482759]
loss in round 30 is  [0.33333333]
loss in round 31 is  [0.32258065]
loss in round 32 is  [0.46875]
loss in round 33 is  [

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss4.csv',loss41, delimiter=',')

In [138]:
loss51=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss5,m3=vimulticlassalgo(x,y,400)
    loss51.append(loss5)

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.]
loss in round 6 is  [0.]
loss in round 7 is  [0.]
loss in round 8 is  [0.]
loss in round 9 is  [0.]
loss in round 10 is  [0.]
loss in round 11 is  [0.]
loss in round 12 is  [0.08333333]
loss in round 13 is  [0.]
loss in round 14 is  [0.]
loss in round 15 is  [0.13333333]
loss in round 16 is  [0.25]
loss in round 17 is  [0.17647059]
loss in round 18 is  [0.16666667]
loss in round 19 is  [0.10526316]
loss in round 20 is  [0.1]
loss in round 21 is  [0.14285714]
loss in round 22 is  [0.09090909]
loss in round 23 is  [0.08695652]
loss in round 24 is  [0.125]
loss in round 25 is  [0.16]
loss in round 26 is  [0.07692308]
loss in round 27 is  [0.07407407]
loss in round 28 is  [0.17857143]
loss in round 29 is  [0.17241379]
loss in round 30 is  [0.16666667]
loss in round 31 is  [0.25806452]
loss in round 32 is  [0.25]
loss in round 33 is  [0.24242424]
loss in round 34 is  

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss5.csv',loss51, delimiter=',')

In [139]:
loss61=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss6,m4=mira(x,y,400)
    loss61.append(loss6)

loss in round 1 is  [0.]
loss in round 2 is  [1.]
loss in round 3 is  [0.33333333]
loss in round 4 is  [0.25]
loss in round 5 is  [0.4]
loss in round 6 is  [0.33333333]
loss in round 7 is  [0.57142857]
loss in round 8 is  [0.625]
loss in round 9 is  [0.77777778]
loss in round 10 is  [0.7]
loss in round 11 is  [0.63636364]
loss in round 12 is  [0.58333333]
loss in round 13 is  [0.69230769]
loss in round 14 is  [0.71428571]
loss in round 15 is  [0.8]
loss in round 16 is  [0.8125]
loss in round 17 is  [0.70588235]
loss in round 18 is  [0.77777778]
loss in round 19 is  [1.05263158]
loss in round 20 is  [0.85]
loss in round 21 is  [0.9047619]
loss in round 22 is  [0.90909091]
loss in round 23 is  [1.]
loss in round 24 is  [0.79166667]
loss in round 25 is  [0.68]
loss in round 26 is  [0.65384615]
loss in round 27 is  [0.66666667]
loss in round 28 is  [0.64285714]
loss in round 29 is  [0.62068966]
loss in round 30 is  [0.7]
loss in round 31 is  [0.61290323]
loss in round 32 is  [0.71875]
loss

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each100_new/loss6.csv',loss61, delimiter=',')

In [140]:
w1_synthetic=[]
loss11=[]
for i in range(500):
    x=np.array(data200)
    y=np.array(data200.sample(axis='columns'))
    loss1,w1=widrowhoff(x,y,400,0.001)
    loss11.append(loss1)
    w1_synthetic.append(w1)

loss in round 1 is  4.0
loss in round 2 is  3.638319999999995
loss in round 3 is  3.5827622023999974
loss in round 4 is  2.800780626215058
loss in round 5 is  2.9767064800220857
loss in round 6 is  2.9953272387553533
loss in round 7 is  3.4173200840130318
loss in round 8 is  2.927209898887937
loss in round 9 is  3.01105082980281
loss in round 10 is  3.3030492731888033
loss in round 11 is  3.1545361269132672
loss in round 12 is  3.1383139279646364
loss in round 13 is  3.1473851152153105
loss in round 14 is  3.370059785193656
loss in round 15 is  3.3058662980363134
loss in round 16 is  3.2584024964639995
loss in round 17 is  3.0387467619012067
loss in round 18 is  3.125011287543364
loss in round 19 is  2.9712745084590964
loss in round 20 is  2.9798855468308734
loss in round 21 is  3.042726484394548
loss in round 22 is  2.974142845104901
loss in round 23 is  2.9400309165129888
loss in round 24 is  2.9048104587756547
loss in round 25 is  2.9843473248739643
loss in round 26 is  2.9892838387

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss1.csv',loss11, delimiter=',')
np.savetxt('/home/22n0457/each200_new/w1_synthetic.csv',w1_synthetic, delimiter=',')

In [141]:
w2_synthetic=[]
b2_synthetic=[]
loss21=[]
for i in range(500):
    x=np.array(data200)
    y=np.array(data200.sample(axis='columns'))
    loss2,w2,b2=pranking(x,y,400)
    loss21.append(loss2)
    w2_synthetic.append(w2)
    b2_synthetic.append(b2)

loss in round 1 is  0.0
loss in round 2 is  0.5
loss in round 3 is  1.3333333333333333
loss in round 4 is  1.75
loss in round 5 is  2.0
loss in round 6 is  2.8333333333333335
loss in round 7 is  2.4285714285714284
loss in round 8 is  2.5
loss in round 9 is  2.5555555555555554
loss in round 10 is  3.2
loss in round 11 is  2.5454545454545454
loss in round 12 is  2.5833333333333335
loss in round 13 is  2.6153846153846154
loss in round 14 is  2.5714285714285716
loss in round 15 is  2.6
loss in round 16 is  2.375
loss in round 17 is  2.235294117647059
loss in round 18 is  2.5
loss in round 19 is  2.3684210526315788
loss in round 20 is  2.25
loss in round 21 is  2.9523809523809526
loss in round 22 is  3.0454545454545454
loss in round 23 is  3.0434782608695654
loss in round 24 is  3.1666666666666665
loss in round 25 is  2.28
loss in round 26 is  2.1923076923076925
loss in round 27 is  2.259259259259259
loss in round 28 is  2.5357142857142856
loss in round 29 is  2.8620689655172415
loss in rou

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss2.csv',loss21, delimiter=',')
np.savetxt('/home/22n0457/each200_new/w2_synthetic.csv',w2_synthetic, delimiter=',')
#np.savetxt('/home/22n0457/each200_new/b2_synthetic.csv',b2_synthetic, delimiter=',')

In [142]:
loss31=[]
for i in range(500):
    x=np.array(data200)
    y=np.array(data200.sample(axis='columns'))
    loss3,m1=uniformmulticlassalgo(x,y,400)
    loss31.append(loss3)
    np.savetxt(f'/home/22n0457/each200_new/m1_synthetic___{i}.csv',m1, delimiter=',')

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.8]
loss in round 6 is  [0.66666667]
loss in round 7 is  [0.28571429]
loss in round 8 is  [0.25]
loss in round 9 is  [0.22222222]
loss in round 10 is  [0.2]
loss in round 11 is  [0.18181818]
loss in round 12 is  [0.]
loss in round 13 is  [0.]
loss in round 14 is  [0.]
loss in round 15 is  [0.]
loss in round 16 is  [0.]
loss in round 17 is  [0.11764706]
loss in round 18 is  [0.]
loss in round 19 is  [0.15789474]
loss in round 20 is  [0.15]
loss in round 21 is  [0.04761905]
loss in round 22 is  [0.04545455]
loss in round 23 is  [0.17391304]
loss in round 24 is  [0.16666667]
loss in round 25 is  [0.28]
loss in round 26 is  [0.15384615]
loss in round 27 is  [0.14814815]
loss in round 28 is  [0.14285714]
loss in round 29 is  [0.13793103]
loss in round 30 is  [0.13333333]
loss in round 31 is  [0.12903226]
loss in round 32 is  [0.125]
loss in round 33 is  [0.18181818]
loss

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss3.csv',loss31, delimiter=',')

In [143]:
loss41=[]
for i in range(500):
    x=np.array(data200)
    y=np.array(data200.sample(axis='columns'))
    loss4,m2=worstmulticlassalgo(x,y,400)
    loss41.append(loss4)
    np.savetxt(f'/home/22n0457/each200_new/m2_synthetic___{i}.csv',m2, delimiter=',')

loss in round 1 is  [5.]
loss in round 2 is  [4.5]
loss in round 3 is  [0.66666667]
loss in round 4 is  [0.5]
loss in round 5 is  [0.8]
loss in round 6 is  [0.66666667]
loss in round 7 is  [0.57142857]
loss in round 8 is  [0.5]
loss in round 9 is  [0.55555556]
loss in round 10 is  [0.5]
loss in round 11 is  [0.45454545]
loss in round 12 is  [0.41666667]
loss in round 13 is  [0.38461538]
loss in round 14 is  [0.35714286]
loss in round 15 is  [0.33333333]
loss in round 16 is  [0.5]
loss in round 17 is  [0.76470588]
loss in round 18 is  [0.66666667]
loss in round 19 is  [1.63157895]
loss in round 20 is  [1.55]
loss in round 21 is  [1.47619048]
loss in round 22 is  [1.]
loss in round 23 is  [0.95652174]
loss in round 24 is  [0.91666667]
loss in round 25 is  [1.]
loss in round 26 is  [0.65384615]
loss in round 27 is  [0.62962963]
loss in round 28 is  [0.75]
loss in round 29 is  [0.86206897]
loss in round 30 is  [0.83333333]
loss in round 31 is  [0.80645161]
loss in round 32 is  [0.78125]
lo

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss4.csv',loss41, delimiter=',')

In [144]:
loss51=[]
for i in range(500):
    x=np.array(data200)
    y=np.array(data200.sample(axis='columns'))
    loss5,m3=vimulticlassalgo(x,y,400)
    loss51.append(loss5)
    np.savetxt(f'/home/22n0457/each200_new/m3_synthetic___{i}.csv',m3, delimiter=',')

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.]
loss in round 6 is  [0.]
loss in round 7 is  [0.]
loss in round 8 is  [0.]
loss in round 9 is  [0.]
loss in round 10 is  [0.]
loss in round 11 is  [0.]
loss in round 12 is  [0.]
loss in round 13 is  [0.]
loss in round 14 is  [0.42857143]
loss in round 15 is  [0.2]
loss in round 16 is  [0.1875]
loss in round 17 is  [0.17647059]
loss in round 18 is  [0.16666667]
loss in round 19 is  [0.15789474]
loss in round 20 is  [0.15]
loss in round 21 is  [0.28571429]
loss in round 22 is  [0.27272727]
loss in round 23 is  [0.26086957]
loss in round 24 is  [0.25]
loss in round 25 is  [0.24]
loss in round 26 is  [0.23076923]
loss in round 27 is  [0.66666667]
loss in round 28 is  [0.64285714]
loss in round 29 is  [0.62068966]
loss in round 30 is  [0.6]
loss in round 31 is  [0.58064516]
loss in round 32 is  [0.75]
loss in round 33 is  [0.72727273]
loss in round 34 is  [0.58823529]

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss5.csv',loss51, delimiter=',')

In [145]:
loss61=[]
for i in range(500):
    x=np.array(data100)
    y=np.array(data100.sample(axis='columns'))
    loss6,m4=mira(x,y,400)
    loss61.append(loss6)
    np.savetxt(f'/home/22n0457/each200_new/m4_synthetic___{i}.csv',m4, delimiter=',')

loss in round 1 is  [0.]
loss in round 2 is  [0.]
loss in round 3 is  [0.]
loss in round 4 is  [0.]
loss in round 5 is  [0.]
loss in round 6 is  [0.33333333]
loss in round 7 is  [0.57142857]
loss in round 8 is  [0.875]
loss in round 9 is  [0.66666667]
loss in round 10 is  [0.6]
loss in round 11 is  [0.90909091]
loss in round 12 is  [1.08333333]
loss in round 13 is  [1.]
loss in round 14 is  [0.71428571]
loss in round 15 is  [0.46666667]
loss in round 16 is  [0.5]
loss in round 17 is  [0.58823529]
loss in round 18 is  [0.66666667]
loss in round 19 is  [0.57894737]
loss in round 20 is  [0.6]
loss in round 21 is  [0.57142857]
loss in round 22 is  [0.68181818]
loss in round 23 is  [0.73913043]
loss in round 24 is  [0.79166667]
loss in round 25 is  [0.6]
loss in round 26 is  [0.73076923]
loss in round 27 is  [0.62962963]
loss in round 28 is  [0.64285714]
loss in round 29 is  [0.68965517]
loss in round 30 is  [0.63333333]
loss in round 31 is  [0.70967742]
loss in round 32 is  [0.5625]
loss i

KeyboardInterrupt: 

In [ ]:
np.savetxt('/home/22n0457/each200_new/loss6.csv',loss61, delimiter=',')